In [1]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
from keras.backend.tensorflow_backend import set_session
set_session(tf.Session(config=config))
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
 

Using TensorFlow backend.


In [2]:

dogs=pd.read_csv("/home/ramreddyy/Documents/dog_breed/labels.csv")


In [3]:
a1=dogs['breed'].value_counts().index.tolist()


In [4]:
from keras.preprocessing import image
def load(inpx,inpy):
    x=[]
    y=[]
    for i in range(inpx.shape[0]):
        if inpx.iloc[i][1] in a1[:5]:
            imgg=image.load_img("/home/ramreddyy/Documents/dog_breed/train/"+inpx.iloc[i][0]+".jpg",target_size=(224,224))
            arr=image.img_to_array(imgg)
            x.append(arr/255)
            y.append(inpy.iloc[i])
    x=np.asarray(x)
    y=np.asarray(y)
    one_hot = pd.get_dummies(y, sparse = True)
    one_hot_labels = np.asarray(one_hot)
    return x,one_hot_labels
    
a,y=load(inpx=dogs,inpy=dogs.breed)


In [7]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
for train_index, test_index in sss.split(a,y):
    x_train, x_test = a[train_index], a[test_index]
    y_train, y_test = y[train_index], y[test_index]
def createModel():
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(224,224,3)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(8, 8)))
    model.add(Dropout(0.1))
 
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(4, 4)))
    model.add(Dropout(0.1))
 
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
 
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax'))
     
    return model

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
model2 = createModel()
 
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
save=keras.callbacks.ModelCheckpoint('/home/ramreddyy/model_save', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
tensor=keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=16, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)


batch_size = 64
epochs = 200
datagen = ImageDataGenerator(
                            zoom_range=0.2,
                            rotation_range=30,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            horizontal_flip=True,
                            vertical_flip=False) 

history2 = model2.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                              steps_per_epoch=int(np.ceil(x_train.shape[0] / float(batch_size))),
                              epochs=epochs,
                              validation_data=(x_test, y_test),
                              workers=4,
                               callbacks=[save,tensor])
 


Epoch 1/200
8/8 [==============================] - 3s 416ms/step - loss: 1.6230 - acc: 0.1894 - val_loss: 1.6083 - val_acc: 0.2034
Epoch 2/200
8/8 [==============================] - 3s 345ms/step - loss: 1.6099 - acc: 0.1960 - val_loss: 1.6071 - val_acc: 0.1949
Epoch 3/200
8/8 [==============================] - 3s 365ms/step - loss: 1.6079 - acc: 0.2079 - val_loss: 1.6054 - val_acc: 0.2119
Epoch 4/200
8/8 [==============================] - 3s 359ms/step - loss: 1.6116 - acc: 0.2428 - val_loss: 1.6048 - val_acc: 0.2119
Epoch 5/200
8/8 [==============================] - 3s 350ms/step - loss: 1.6079 - acc: 0.2211 - val_loss: 1.6041 - val_acc: 0.2373
Epoch 6/200
8/8 [==============================] - 3s 357ms/step - loss: 1.6016 - acc: 0.2922 - val_loss: 1.5772 - val_acc: 0.3644
Epoch 7/200
8/8 [==============================] - 3s 375ms/step - loss: 1.5850 - acc: 0.2652 - val_loss: 1.5906 - val_acc: 0.2203
Epoch 8/200
8/8 [==============================] - 3s 373ms/step - loss: 1.5402 - a

8/8 [==============================] - 3s 358ms/step - loss: 0.6177 - acc: 0.7178 - val_loss: 1.1806 - val_acc: 0.5678
Epoch 64/200
8/8 [==============================] - 3s 374ms/step - loss: 0.6558 - acc: 0.7224 - val_loss: 1.0393 - val_acc: 0.5932
Epoch 65/200
8/8 [==============================] - 3s 356ms/step - loss: 0.6234 - acc: 0.7619 - val_loss: 1.0463 - val_acc: 0.5847
Epoch 66/200
8/8 [==============================] - 3s 354ms/step - loss: 0.6354 - acc: 0.7487 - val_loss: 1.2407 - val_acc: 0.5593
Epoch 67/200
8/8 [==============================] - 3s 355ms/step - loss: 0.6374 - acc: 0.7269 - val_loss: 1.1355 - val_acc: 0.5932
Epoch 68/200
8/8 [==============================] - 3s 362ms/step - loss: 0.6139 - acc: 0.7586 - val_loss: 1.2000 - val_acc: 0.5424
Epoch 69/200
8/8 [==============================] - 3s 361ms/step - loss: 0.5759 - acc: 0.7467 - val_loss: 1.1519 - val_acc: 0.5932
Epoch 70/200
8/8 [==============================] - 3s 398ms/step - loss: 0.6558 - acc: 0

Epoch 125/200
8/8 [==============================] - 3s 357ms/step - loss: 0.2544 - acc: 0.9053 - val_loss: 1.3990 - val_acc: 0.6186
Epoch 126/200
8/8 [==============================] - 3s 362ms/step - loss: 0.2801 - acc: 0.8829 - val_loss: 1.2634 - val_acc: 0.6525
Epoch 127/200
8/8 [==============================] - 3s 350ms/step - loss: 0.3264 - acc: 0.8869 - val_loss: 1.4377 - val_acc: 0.5847
Epoch 128/200
8/8 [==============================] - 3s 405ms/step - loss: 0.2726 - acc: 0.8882 - val_loss: 1.5806 - val_acc: 0.5763
Epoch 129/200
8/8 [==============================] - 3s 410ms/step - loss: 0.2618 - acc: 0.9119 - val_loss: 1.3785 - val_acc: 0.6441
Epoch 130/200
8/8 [==============================] - 3s 370ms/step - loss: 0.3228 - acc: 0.8823 - val_loss: 1.2101 - val_acc: 0.6441
Epoch 131/200
8/8 [==============================] - 3s 360ms/step - loss: 0.3097 - acc: 0.8843 - val_loss: 1.0616 - val_acc: 0.7203
Epoch 132/200
8/8 [==============================] - 3s 362ms/step - 

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=[8,6])
plt.plot(history2.history['loss'],'r',linewidth=3.0)
plt.plot(history2.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)


plt.figure(figsize=[8,6])
plt.plot(history2.history['acc'],'r',linewidth=3.0)
plt.plot(history2.history['val_acc'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)
plt.show()